This notebook is for experimenting with a dual-CNN model. One CNN (which I have already fine-tuned a little) will make predictions on the 6 labels. The second CNN will take the output of running the first CNN by running the 2 closest slices (in the same CT scan) from above a below, meaning that the second CNN takes in 5 input vectora

In [1]:
import pydicom as dcm
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import sys
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from matplotlib import pyplot as plt
import math
import csv
from joblib import Parallel, delayed

In [2]:
train_img_dir = 'rsna-intracranial-hemorrhage-detection/stage_2_train_imgs/'
train_label_path = 'rsna-intracranial-hemorrhage-detection/train_labels.csv'
train_ct_path = 'rsna-intracranial-hemorrhage-detection/train_ct_scans.csv'
train_coord_path = 'rsna-intracranial-hemorrhage-detection/train_ct_coords.csv'

test_img_dir = 'rsna-intracranial-hemorrhage-detection/stage_2_test_imgs/'

keras.mixed_precision.set_global_policy('mixed_float16')
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: A100-SXM4-40GB, compute capability 8.0


In [3]:
train_img_ct = {} # scan index : list of image IDs in the scan
train_img_ct_ind = {} #image ID : {"ct_ind": index of the CT scan this image belongs to (key in train_img_ct), "ind": index in the list of slices}

train_img_coords = pd.read_csv(train_coord_path, index_col=0, names=['x','y','z'])
with open(train_ct_path) as scans:
    reader = csv.reader(scans, delimiter=',')
    i = 0
    imgs = None
    for row in tqdm(list(reader)):
        imgs = row[1:]
        imgs.sort(key=lambda x: train_img_coords.loc[x]['z'])
        for slice_ind, img_id in enumerate(imgs):
            train_img_ct_ind[img_id] = {'ct_ind': i, 'ind': slice_ind}
        train_img_ct[i] = imgs
        i += 1

100%|██████████| 18938/18938 [00:48<00:00, 393.46it/s]


In [4]:
def get_img_tensor(img_path):
    return tf.convert_to_tensor(np.asarray(Image.open(img_path), dtype=np.float32) / 255.)


class RSNASequence(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = [self.y[img_id.split('/')[-1].split('.')[0]] for img_id in batch_x]
        
        return (tf.stack([get_img_tensor(img_path) for img_path in batch_x], axis=0), 
               tf.convert_to_tensor(batch_y))
    
    def on_epoch_end(self):
        ind = np.random.choice(list(range(len(os.listdir(train_img_dir)))), size=train_cutoff, replace=False)
        self.x = [train_img_dir + img_name for img_name in np.array(os.listdir(train_img_dir))[ind]]
    

In [17]:
def get_nearby_slices(img_dir, img_name, n_slices):
    '''
    will retrieve n_slices slices from BOTH above and below the given image. If there is not enough space, it will
    add more slices either below (if the image is near the top of the scan) or above (if the image is near the bottom of the scan).
    Exactly 2*n_slices + 1 images will be returned
    '''
    img_id = img_name.split('.')[0]
    ct_ind, ind = train_img_ct_ind[img_id]['ct_ind'], train_img_ct_ind[img_id]['ind']
    ct = train_img_ct[ct_ind]
    n = len(ct)
    low, high = ind-n_slices, ind+n_slices
    if low < 0:
        high += abs(low)
        low = 0
    elif high >= n:
        low -= high - (n-1)
        high = n-1
    
    return [get_img_tensor(img_dir+img_name) for img_id in ct[low:high+1]]
    
    
    

In [6]:
labels = pd.read_csv(train_label_path)
labels = {l[0]: l[1:].astype(np.int8) for l in labels.to_numpy()}

In [7]:
train_cutoff = 320 #training the whole dataset takes ~9 hours, so we cut it short for proof-of-concept purposes.
ind = np.random.choice(list(range(len(os.listdir(train_img_dir)))), size=train_cutoff, replace=False)
train_sequence = RSNASequence([train_img_dir + img_name for img_name in np.array(os.listdir(train_img_dir))[ind]], labels, 32)

In [8]:
img = train_img_ct[0][0] + '.png' #np.random.choice(os.listdir(train_img_dir))
slices = get_nearby_slices(train_img_dir, img, 2)
t = tf.convert_to_tensor(slices)
t.shape

2021-12-23 00:04:37.887664: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-23 00:04:38.422031: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38444 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


TensorShape([5])

In [9]:
model = keras.models.load_model('reproduce_training_2/model.h5')

In [10]:
extractor = keras.models.Sequential(model.layers[:-1])
extractor.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 16, 16, 2048)      42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 42,658,176
Trainable params: 42,552,832
Non-trainable params: 105,344
_________________________________________________________________


In [11]:
slice_window = 9

model = keras.models.Sequential()
model.add(Conv2D(16, 3, activation='relu', input_shape=(2*slice_window+1,2048,1)))
model.add(Conv2D(32, 4, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256, 3, activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics=['binary_accuracy', 
                       keras.metrics.AUC(multi_label=True, num_labels=6, from_logits=False),
                       keras.metrics.Precision(), keras.metrics.Recall()],
             optimizer=keras.optimizers.Adam(learning_rate=1e-3))

cp_callback = keras.callbacks.ModelCheckpoint(filepath='experiment-1-checkpoints/checkpoint.ckpt',
                                                 save_weights_only=False,
                                                 verbose=1)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 17, 2046, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 2043, 32)      8224      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 1021, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 1019, 256)      73984     
                                                                 
 global_average_pooling2d (G  (None, 256)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 6)                

In [21]:
class RSNASequence2(keras.utils.Sequence):
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = [self.y[img_name.split('.')[0]] for img_name in batch_x]
        
        imgs = [tf.convert_to_tensor(get_nearby_slices(train_img_dir, img_name, slice_window)) for img_name in batch_x]
        # feats = [tf.convert_to_tensor([ for slice in batch]) for batch in batch_x]
        # return (tf.convert_to_tensor([ [features[img_name.split('.')[0]] for img_name in batch] for batch in batch_x]), 
                                     # tf.convert_to_tensor(batch_y))
        return (tf.convert_to_tensor([extractor.predict(im) for im in imgs]), tf.convert_to_tensor(batch_y))
    

In [22]:
train_cutoff = 25600 #training the whole dataset takes ~9 hours, so we cut it short for proof-of-concept purposes.
ind = np.random.choice(list(range(len(os.listdir(train_img_dir)))), size=train_cutoff, replace=False)
train_subset = np.array(os.listdir(train_img_dir))[ind]
# features = {}
# for img_name in tqdm(train_subset):
    # features[img_name.split('.')[0]] = extractor.predict(tf.expand_dims(get_img_tensor(train_img_dir+img_name), axis=0))
# Parallel(n_jobs=10)(delayed(save_features)(img_name) for img_name in tqdm(train_subset))

train_sequence = RSNASequence2(train_subset, labels, 32)


In [24]:
model.fit(x=train_sequence, epochs=3, callbacks=[cp_callback])

Epoch 1/3
 15/800 [..............................] - ETA: 2:06:45 - loss: 0.4184 - binary_accuracy: 0.9062 - auc: 0.5449 - precision: 0.0517 - recall: 0.0361

KeyboardInterrupt: 